In [11]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json

In [12]:
with open("../data/credentials.json") as credentials_file:
        credentials = json.load(credentials_file)

In [13]:
client_id = credentials["client_id"]
client_secret = credentials["client_secret"]

In [14]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Defining a function to get track and audio feature info for spotify top tracks playlist

In [ ]:
def get_playlist_data(spotify, playlist_id):
    playlist = spotify.playlist(playlist_id)
    playlist_name = playlist['name']
    total_tracks = playlist['tracks']['total']
    
    ## Spotify API usually has a max of 50-100 tracks it will retrieve in batches.
    ## This handles pagination to return all tracks from a playlist.
    limit = 100
    offset = 0
    track_data = []
    while offset < total_tracks:
        results = spotify.playlist_tracks(playlist_id, offset=offset, limit=limit)
        tracks = results['items'] ## First API call made to retrieve track info
        for track in tracks: 
            track_info = track['track']
            track_id = track_info['id']
            track_name = track_info['name']
            track_artists = ', '.join([artist['name'] for artist in track_info['artists']])
            track_album = track_info['album']['name']
            track_popularity = track_info['popularity']
            track_url = track_info['external_urls']['spotify']
            play_count = track_info['album']['total_tracks']
            
            track_details = spotify.track(track_id) 
            explicit = track_details['explicit']
            ## Second API call for audio features.
            features = spotify.audio_features(track_id)[0] 
            acousticness = features['acousticness']
            danceability = features['danceability']
            loudness = features['loudness']
            valence = features['valence']
            energy = features['energy']
            liveness = features['liveness']
            tempo = features['tempo']
            instrumentalness = features['instrumentalness']
            duration_ms = features['duration_ms']
            ## Final API call to retrieve genres
            genres = spotify.artist(track_info['artists'][0]['id'])['genres'] 
            
            album_id = track_info['album']['id']
            album = spotify.album(album_id)
            release_year = album['release_date'][:4]
            
            ## Appending lists to track_date list 
            track_data.append([track_name, track_artists, track_album, track_popularity, play_count, track_url,
                               acousticness, danceability, loudness, valence, energy, liveness,tempo,explicit, instrumentalness,
                               duration_ms, genres, release_year])

        offset += limit
    
    ## Setting preferred column names for the final dataframe output from API calls  
    columns = ['Title', 'Artist', 'Album', 'Popularity', 'Play Count', 'Track URL', 'Acousticness',
               'Danceability', 'Loudness', 'Valence', 'Energy', 'Liveness','Tempo','Explicit', 'Instrumentalness', 'Duration (ms)', 'Genres',
               'Release Year']
    
    df = pd.DataFrame(track_data, columns=columns)
    return df

In [ ]:
## Call function and change playlist ID after each run to get next playlist.

In [ ]:
playlist_id = '06FpomWnZbAjMeuL6YLZ4i'
playlist_1950 = get_playlist_data(spotify,playlist_id)

In [ ]:
playlist_1950

In [ ]:
##drop releaes year and replace with new column named the same and change all rows to
##top hits year. Save to csv file
playlist_1950=playlist1950.drop(columns=['Release Year'])
playlist1950['Release Year']=1950
playlist1950.to_csv('../data/playlist.csv')
